# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# Only necessary when executing this notebook on the Udacity cloud
!pip -q install ../python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


In [2]:
from unityagents import UnityEnvironment
import numpy as np
import os
import time

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [3]:
# Standalone machine
#env = UnityEnvironment( os.path.join( os.environ['HOME'], 'Python/rl/udadrl/data/Banana_Linux/Banana.x86_64' ) )

# Udacity cloud
env = UnityEnvironment( file_name="/data/Banana_Linux_NoVis/Banana.x86_64" )

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
from dql_agent import Agent

agent = Agent(state_size, action_size, 19711108)

In [7]:

for epc in range(4000):

    env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0     # initialize the score

    start_time = time.time()

    while True:
        action = agent.act(state)        # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        agent.step(state, action, reward, next_state, done)
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    stop_time = time.time()

    print( "Epoch:{:>5}; Score: {:>5}; Execution time: {:.4f}".format( epc+1, score, stop_time - start_time ) )


Epoch:    1, Score:   0.0, Execution time: 2.1981
Epoch:    2, Score:   1.0, Execution time: 0.9358
Epoch:    3, Score:   0.0, Execution time: 0.9293
Epoch:    4, Score:   0.0, Execution time: 0.9102
Epoch:    5, Score:  -2.0, Execution time: 0.8764
Epoch:    6, Score:   0.0, Execution time: 0.8554
Epoch:    7, Score:   1.0, Execution time: 0.8436
Epoch:    8, Score:   0.0, Execution time: 0.8527
Epoch:    9, Score:   0.0, Execution time: 0.8613
Epoch:   10, Score:   0.0, Execution time: 0.8478
Epoch:   11, Score:   0.0, Execution time: 0.8758
Epoch:   12, Score:   0.0, Execution time: 0.8522
Epoch:   13, Score:   0.0, Execution time: 0.8545
Epoch:   14, Score:  -1.0, Execution time: 0.8835
Epoch:   15, Score:   0.0, Execution time: 0.8676
Epoch:   16, Score:   0.0, Execution time: 0.8568
Epoch:   17, Score:   0.0, Execution time: 0.8541
Epoch:   18, Score:  -2.0, Execution time: 0.8890
Epoch:   19, Score:   0.0, Execution time: 0.8912
Epoch:   20, Score:   1.0, Execution time: 0.8565


Epoch:  165, Score:   0.0, Execution time: 0.9030
Epoch:  166, Score:   0.0, Execution time: 0.9054
Epoch:  167, Score:   0.0, Execution time: 0.8849
Epoch:  168, Score:   0.0, Execution time: 0.8836
Epoch:  169, Score:   0.0, Execution time: 0.8977
Epoch:  170, Score:   1.0, Execution time: 0.8986
Epoch:  171, Score:   0.0, Execution time: 0.9129
Epoch:  172, Score:  -1.0, Execution time: 0.9279
Epoch:  173, Score:   0.0, Execution time: 0.9078
Epoch:  174, Score:   0.0, Execution time: 0.9106
Epoch:  175, Score:   0.0, Execution time: 0.9131
Epoch:  176, Score:   0.0, Execution time: 0.9397
Epoch:  177, Score:  -1.0, Execution time: 0.9048
Epoch:  178, Score:  -2.0, Execution time: 0.9173
Epoch:  179, Score:  -1.0, Execution time: 0.8983
Epoch:  180, Score:   0.0, Execution time: 0.9231
Epoch:  181, Score:   0.0, Execution time: 0.9353
Epoch:  182, Score:   1.0, Execution time: 0.9083
Epoch:  183, Score:   0.0, Execution time: 0.9026
Epoch:  184, Score:   0.0, Execution time: 0.8856


Epoch:  329, Score:   0.0, Execution time: 0.9195
Epoch:  330, Score:   1.0, Execution time: 0.9860
Epoch:  331, Score:   2.0, Execution time: 0.8984
Epoch:  332, Score:  -1.0, Execution time: 0.9126
Epoch:  333, Score:   3.0, Execution time: 0.9184
Epoch:  334, Score:   2.0, Execution time: 0.9275
Epoch:  335, Score:   1.0, Execution time: 0.9203
Epoch:  336, Score:   1.0, Execution time: 0.9197
Epoch:  337, Score:   1.0, Execution time: 0.9206
Epoch:  338, Score:   0.0, Execution time: 0.9419
Epoch:  339, Score:  -1.0, Execution time: 0.9564
Epoch:  340, Score:   0.0, Execution time: 0.9287
Epoch:  341, Score:   4.0, Execution time: 0.9160
Epoch:  342, Score:   0.0, Execution time: 0.9122
Epoch:  343, Score:   0.0, Execution time: 0.9209
Epoch:  344, Score:   0.0, Execution time: 0.9457
Epoch:  345, Score:  -1.0, Execution time: 0.9349
Epoch:  346, Score:   1.0, Execution time: 0.9264
Epoch:  347, Score:   3.0, Execution time: 0.9329
Epoch:  348, Score:   0.0, Execution time: 0.9018


Epoch:  493, Score:  18.0, Execution time: 0.9555
Epoch:  494, Score:  12.0, Execution time: 0.9648
Epoch:  495, Score:   5.0, Execution time: 0.9439
Epoch:  496, Score:   9.0, Execution time: 0.9550
Epoch:  497, Score:   6.0, Execution time: 1.0065
Epoch:  498, Score:  10.0, Execution time: 0.9459
Epoch:  499, Score:   7.0, Execution time: 0.9369
Epoch:  500, Score:   7.0, Execution time: 0.9636
Epoch:  501, Score:  10.0, Execution time: 0.9675
Epoch:  502, Score:   4.0, Execution time: 0.9713
Epoch:  503, Score:  10.0, Execution time: 1.0520
Epoch:  504, Score:  11.0, Execution time: 0.9462
Epoch:  505, Score:   7.0, Execution time: 0.9414
Epoch:  506, Score:  18.0, Execution time: 0.9478
Epoch:  507, Score:  13.0, Execution time: 0.9455
Epoch:  508, Score:   6.0, Execution time: 0.9533
Epoch:  509, Score:   5.0, Execution time: 0.9695
Epoch:  510, Score:   4.0, Execution time: 0.9509
Epoch:  511, Score:   4.0, Execution time: 0.9453
Epoch:  512, Score:  10.0, Execution time: 0.9547


Epoch:  657, Score:  18.0, Execution time: 0.9954
Epoch:  658, Score:  15.0, Execution time: 0.9831
Epoch:  659, Score:   7.0, Execution time: 1.0054
Epoch:  660, Score:   3.0, Execution time: 0.9671
Epoch:  661, Score:   9.0, Execution time: 0.9877
Epoch:  662, Score:  16.0, Execution time: 0.9332
Epoch:  663, Score:  12.0, Execution time: 0.9433
Epoch:  664, Score:   6.0, Execution time: 0.9398
Epoch:  665, Score:  15.0, Execution time: 0.9535
Epoch:  666, Score:   9.0, Execution time: 0.9476
Epoch:  667, Score:  14.0, Execution time: 0.9661
Epoch:  668, Score:   7.0, Execution time: 0.9501
Epoch:  669, Score:  14.0, Execution time: 0.9529
Epoch:  670, Score:  12.0, Execution time: 0.9446
Epoch:  671, Score:  16.0, Execution time: 0.9317
Epoch:  672, Score:   8.0, Execution time: 0.9426
Epoch:  673, Score:  16.0, Execution time: 0.9423
Epoch:  674, Score:  18.0, Execution time: 0.9318
Epoch:  675, Score:  12.0, Execution time: 0.9460
Epoch:  676, Score:  16.0, Execution time: 0.9620


...

Epoch: 3281, Score:  11.0, Execution time: 0.9202
Epoch: 3282, Score:  10.0, Execution time: 0.9079
Epoch: 3283, Score:  14.0, Execution time: 0.9124
Epoch: 3284, Score:  15.0, Execution time: 0.9092
Epoch: 3285, Score:  13.0, Execution time: 0.9199
Epoch: 3286, Score:  11.0, Execution time: 0.9158
Epoch: 3287, Score:   9.0, Execution time: 0.9108
Epoch: 3288, Score:   1.0, Execution time: 0.9203
Epoch: 3289, Score:  12.0, Execution time: 0.9348
Epoch: 3290, Score:   7.0, Execution time: 0.9437
Epoch: 3291, Score:  17.0, Execution time: 0.9191
Epoch: 3292, Score:  10.0, Execution time: 0.9183
Epoch: 3293, Score:  15.0, Execution time: 0.9093
Epoch: 3294, Score:   7.0, Execution time: 0.9021
Epoch: 3295, Score:  11.0, Execution time: 0.9127
Epoch: 3296, Score:  11.0, Execution time: 0.9659
Epoch: 3297, Score:  13.0, Execution time: 0.9439
Epoch: 3298, Score:  15.0, Execution time: 0.9256
Epoch: 3299, Score:  12.0, Execution time: 0.9291
Epoch: 3300, Score:  13.0, Execution time: 0.9349


Epoch: 3445, Score:  11.0, Execution time: 0.9797
Epoch: 3446, Score:   9.0, Execution time: 0.9378
Epoch: 3447, Score:   8.0, Execution time: 0.9258
Epoch: 3448, Score:  14.0, Execution time: 0.9317
Epoch: 3449, Score:   7.0, Execution time: 0.9244
Epoch: 3450, Score:  14.0, Execution time: 0.9271
Epoch: 3451, Score:  16.0, Execution time: 0.9250
Epoch: 3452, Score:  19.0, Execution time: 0.9182
Epoch: 3453, Score:  18.0, Execution time: 0.9115
Epoch: 3454, Score:  18.0, Execution time: 0.9257
Epoch: 3455, Score:  13.0, Execution time: 0.9118
Epoch: 3456, Score:  14.0, Execution time: 0.9387
Epoch: 3457, Score:  14.0, Execution time: 0.9177
Epoch: 3458, Score:  17.0, Execution time: 0.9099
Epoch: 3459, Score:  15.0, Execution time: 0.9255
Epoch: 3460, Score:  10.0, Execution time: 0.9401
Epoch: 3461, Score:   9.0, Execution time: 0.9249
Epoch: 3462, Score:  11.0, Execution time: 0.9389
Epoch: 3463, Score:  11.0, Execution time: 0.9134
Epoch: 3464, Score:  13.0, Execution time: 0.9109


Epoch: 3609, Score:  13.0, Execution time: 0.9331
Epoch: 3610, Score:   7.0, Execution time: 0.9297
Epoch: 3611, Score:  14.0, Execution time: 0.9435
Epoch: 3612, Score:  12.0, Execution time: 0.9376
Epoch: 3613, Score:  13.0, Execution time: 0.9369
Epoch: 3614, Score:  11.0, Execution time: 0.9465
Epoch: 3615, Score:   8.0, Execution time: 0.9953
Epoch: 3616, Score:   3.0, Execution time: 0.9113
Epoch: 3617, Score:  17.0, Execution time: 0.9191
Epoch: 3618, Score:  14.0, Execution time: 0.9257
Epoch: 3619, Score:  16.0, Execution time: 0.9483
Epoch: 3620, Score:   6.0, Execution time: 0.9447
Epoch: 3621, Score:  15.0, Execution time: 0.9661
Epoch: 3622, Score:  11.0, Execution time: 0.9335
Epoch: 3623, Score:  15.0, Execution time: 0.9304
Epoch: 3624, Score:   5.0, Execution time: 0.9254
Epoch: 3625, Score:  11.0, Execution time: 0.9325
Epoch: 3626, Score:  18.0, Execution time: 0.9317
Epoch: 3627, Score:   7.0, Execution time: 0.9027
Epoch: 3628, Score:   6.0, Execution time: 0.9209


Epoch: 3773, Score:  11.0, Execution time: 0.9398
Epoch: 3774, Score:  13.0, Execution time: 0.9463
Epoch: 3775, Score:  11.0, Execution time: 0.9503
Epoch: 3776, Score:  13.0, Execution time: 0.9594
Epoch: 3777, Score:  12.0, Execution time: 0.9888
Epoch: 3778, Score:  15.0, Execution time: 0.9951
Epoch: 3779, Score:   8.0, Execution time: 0.9667
Epoch: 3780, Score:  11.0, Execution time: 0.9339
Epoch: 3781, Score:  18.0, Execution time: 0.9408
Epoch: 3782, Score:  15.0, Execution time: 1.0121
Epoch: 3783, Score:  10.0, Execution time: 0.9463
Epoch: 3784, Score:  24.0, Execution time: 0.9632
Epoch: 3785, Score:  15.0, Execution time: 0.9331
Epoch: 3786, Score:   9.0, Execution time: 0.9562
Epoch: 3787, Score:  18.0, Execution time: 0.9642
Epoch: 3788, Score:  10.0, Execution time: 0.9725
Epoch: 3789, Score:  12.0, Execution time: 0.9519
Epoch: 3790, Score:  12.0, Execution time: 0.9331
Epoch: 3791, Score:  13.0, Execution time: 0.9355
Epoch: 3792, Score:  13.0, Execution time: 0.9379


Epoch: 3937, Score:  13.0, Execution time: 0.9432
Epoch: 3938, Score:  10.0, Execution time: 0.9432
Epoch: 3939, Score:  13.0, Execution time: 0.9266
Epoch: 3940, Score:  14.0, Execution time: 0.9480
Epoch: 3941, Score:  14.0, Execution time: 0.9378
Epoch: 3942, Score:   8.0, Execution time: 0.9258
Epoch: 3943, Score:  11.0, Execution time: 0.9265
Epoch: 3944, Score:  18.0, Execution time: 0.9431
Epoch: 3945, Score:  11.0, Execution time: 0.9426
Epoch: 3946, Score:  10.0, Execution time: 0.9398
Epoch: 3947, Score:  14.0, Execution time: 0.9251
Epoch: 3948, Score:  16.0, Execution time: 0.9231
Epoch: 3949, Score:  16.0, Execution time: 0.9392
Epoch: 3950, Score:  18.0, Execution time: 0.9920
Epoch: 3951, Score:   5.0, Execution time: 0.9476
Epoch: 3952, Score:  16.0, Execution time: 0.9449
Epoch: 3953, Score:  12.0, Execution time: 0.9202
Epoch: 3954, Score:  12.0, Execution time: 0.9230
Epoch: 3955, Score:  16.0, Execution time: 0.9706
Epoch: 3956, Score:  18.0, Execution time: 0.9690


In [15]:
import torch

torch.save(agent.qnetwork_local.state_dict(), '20200704a_qnetwork_local_statedict.pth')

torch.save(agent.qnetwork_target.state_dict(), '20200704a_qnetwork_target_statedict.pth')

In [16]:
env.close()